In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import sys
from threading import Thread

import h5py
import hdf5storage # conda install hdf5storage
import ipywidgets as widgets

%config Completer.use_jedi = False

# # required to enable non-blocking interaction:
%gui qt5

from pyphocorehelpers.Filesystem.HDF5.hdf5_file_helpers import h5dump, HDF5_Helper
from pyphocorehelpers.Filesystem.HDF5.silx_hdf5_viewer import hdf5_tree_view_window

from pyphocorehelpers.Filesystem.path_helpers import discover_data_files, get_file_metadata
from pyphocorehelpers.print_helpers import debug_dump_object_member_shapes, print_value_overview_only, pprint, debug_print, print_keys_if_possible
from pyphoplacecellanalysis.PhoPositionalData.load_exported import import_mat_file
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, extract_spike_timeseries
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import _test_save_pipeline_data_to_h5, get_h5_data_keys, save_some_pipeline_data_to_h5, load_pipeline_data_from_h5  #ExportHelpers

from pyphoplacecellanalysis.GUI.PyQtPlot.Params.MatFileBrowser.MatFileBrowser import MatFileBrowser

## Drastically improve Python Tracebacks on exceptions:
# from rich.traceback import install
# install()

In [ ]:
# RoyMaze1:

mat_import_parent_path = Path(r'R:\data\RoyMaze1')
# mat_import_parent_path = Path(r'C:\Share\data\RoyMaze1') # Old one
# mat_import_file = r'C:\Share\data\RoyMaze1\ExportedData.mat'

found_files = MatFileBrowser.discover_mat_files(mat_import_parent_path, recursive=True)
found_files

In [ ]:
# from param import define, field, Factory
# class FilesCollection:




In [2]:
search_parent_path = Path(r'W:\\Data\\Kdiba')
found_default_session_pickle_files = discover_data_files(search_parent_path, glob_pattern='loadedSessPickle.pkl', recursive=True)
found_autoversioned_session_pickle_files = discover_data_files(search_parent_path, glob_pattern='*-loadedSessPickle.pkl', recursive=True)
found_global_computation_results_files = discover_data_files(search_parent_path, glob_pattern='output/*.pkl', recursive=True)

found_files = found_default_session_pickle_files + found_global_computation_results_files + found_autoversioned_session_pickle_files
found_files

[WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-07_11-26-53/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-09_3-23-37/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-12_15-55-31/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/one/2006-6-13_14-42-6/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-07_16-40-19/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-08_15-46-47/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-08_21-16-25/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-09_22-24-40/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-12_16-53-46/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/gor01/two/2006-6-13_15-22-3/loadedSessPickle.pkl'),
 WindowsPath('W:/Data/Kdiba/pin0

In [3]:
# pd.DataFrame({'path': found_files})

get_file_metadata(found_files)

,path,modification_time,creation_time,file_size
0,W:\Data\Kdiba\gor01\one\2006-6-07_11-26-53\loa...,2023-06-06 20:22:33.863387,2023-05-15 16:46:57.057551,17.155477
1,W:\Data\Kdiba\gor01\one\2006-6-08_14-26-15\loa...,2023-06-06 20:29:04.110998,2023-05-26 15:58:19.787755,7.403408
2,W:\Data\Kdiba\gor01\one\2006-6-09_1-22-43\load...,2023-06-06 20:50:58.373352,2023-04-18 19:58:11.555146,21.320790
3,W:\Data\Kdiba\gor01\one\2006-6-09_3-23-37\load...,2023-05-20 00:36:21.068606,2023-05-17 06:27:30.460786,0.173567
4,W:\Data\Kdiba\gor01\one\2006-6-12_15-55-31\loa...,2023-06-06 21:10:27.573423,2023-05-15 16:47:26.161258,13.077574
...,...,...,...,...
87,W:\Data\Kdiba\gor01\one\2006-6-09_1-22-43\2023...,2023-04-24 15:38:43.522915,2023-04-24 15:38:41.974823,0.416536
88,W:\Data\Kdiba\gor01\one\2006-6-09_1-22-43\2023...,2023-05-24 09:24:46.399799,2023-05-24 09:24:46.397799,0.000042
89,W:\Data\Kdiba\vvp01\two\2006-4-11_16-2-46\2023...,2023-06-07 00:33:10.785533,2023-06-07 00:32:43.376257,12.715891
90,W:\Data\Kdiba\vvp01\two\2006-4-12_15-25-59\202...,2023-06-07 00:38:38.327967,2023-06-07 00:38:38.307967,0.006206


In [ ]:
found_file_variables_dict = MatFileBrowser.whos_mat_files(found_files)
print(found_file_variables_dict)

# Silx hdf5 Widgets

In [ ]:
non_hdf5_files = [Path(r'R:/data/RoyMaze1/ExportedData/extrasAnalysis.mat'), Path('R:/data/RoyMaze1/ExportedData/RippleIndexedSpikes.mat')] # , Path(r'R:/data/RoyMaze1/ExportedData/positionAnalysis.mat')
hdf5_found_files = [str(found_file) for found_file in found_files if found_file not in non_hdf5_files]
print(f'hdf5_found_files: {hdf5_found_files}')
win = hdf5_tree_view_window(hdf5_found_files)
win

# Overflow (Not needed, Slix HDF5 widget worked to display/browse the data)

In [ ]:
# Import the positions
t,x,y,speeds,dt,dx,dy = perform_import_positions(mat_import_parent_path=mat_import_parent_path)
# Import the spikes
spike_matrix, spike_cells, num_cells, spike_list, spike_positions_list = perform_import_spikes(t, x, y, mat_import_parent_path=mat_import_parent_path)

ripple_times, ripple_peak_times, ripple_peak_values, behavioral_periods, behavioral_epochs = perform_import_extras(mat_import_parent_path=mat_import_parent_path)

# behavioral_periods = all_results_data['active_processing/behavioral_periods_table']
# print('spike_matrix: {}, spike_cells: {}'.format(np.shape(spike_matrix), np.shape(spike_cells)))
num_periods = np.shape(behavioral_periods)[0]
print('num_periods: {}'.format(num_periods))


# creating the dataset
#date = [i for i in range(2000, 2021)]
 
# Create a figure and a set of subplots
fig, ax = plt.subplots(figsize=(10, 6))
 
# Creating the scatter plot
#ax.scatter(date, value)
 
# Highlighting for a certain period of time
num_ripples = np.shape(ripple_times)[0]
for i in np.arange(num_ripples):
    ax.axvspan(ripple_times[i,0], ripple_times[i,1], alpha=0.3, color="green")

plt.show()

In [ ]:
import silx.gui.hdf5
treeview = silx.gui.hdf5.Hdf5TreeView()
model = treeview.findHdf5TreeModel()
# Avoid the user to drop file in the widget
model.setFileDropEnabled(False)
# Allow the user to reorder files with drag-and-drop
model.setFileMoveEnabled(True)
header = treeview.header()
# Select displayed columns
column_ids = [treeview.findHdf5TreeModel().NAME_COLUMN]
header.setSections(column_ids)
# Do not allow the user to custom visible columns
header.setEnableHideColumnsPopup(False)

## MatFileBrowser.build_browsing_gui(...) Parameter Tree GUI (WORKING, NOT USED)

In [ ]:
tree, app = MatFileBrowser.build_browsing_gui(found_file_variables_dict)
tree.show()

## HDF5_Helper Recurrsion Testing (NOT FINISHED, NOT WORKING, NOT USED)

In [ ]:
# found_file = Path(r'R:/data/RoyMaze1/analysesResults_30-Sep-2021/allEnv.mat')
# found_file = Path(r'R:/data/RoyMaze1/ExportedData/spikesAnalysis.mat')
# found_file = Path(r'R:\data\RoyMaze1\ExportedData\extrasAnalysis.mat')
found_file = Path(r'R:/data/RoyMaze1/ExportedData/spikesTable.mat')

In [ ]:
# import h5py
helper = HDF5_Helper(path=found_file, group='active_processing')
helper.output_tree

In [ ]:
# helper.perform_enumerate(debug_print=True)
helper.perform_enumerate(debug_print=False)
helper.output_tree

In [ ]:
found_file_variables_dict[found_file]

In [ ]:
h5dump(found_file, group='active_processing', enable_print_attributes=False)

## h5py 'visit_item' testing (WORKING, NOT USED):

In [ ]:
def pho_visit_item_fn(name, obj):
    # Recursively visit all objects in this group and subgroups. Like Group.visit(), except your callable should have the signature:
    # callable(name, object) -> None or return value
    # In this case object will be a Group or Dataset instance.
    # if name not in ignored_keys:        
    if '#' in name:
        return None
    else:
        print(f'name: {name}, obj: {obj}')
        # if type(obj) in [h5py._hl.group.Group, h5py._hl.files.File]:
        #     for key in obj.keys():
        #         print(sep,'-',key,':',obj[key])
        #         descend_obj(obj[key],sep=sep+'\t')
        # elif type(obj)==h5py._hl.dataset.Dataset:
        #     for key in obj.attrs.keys():
        #         print(sep+'\t','-',key,':',obj.attrs[key])
            
    
def find_foo(name):
    """ Find first object with 'foo' anywhere in the name """
    if 'foo' in name:
        return name

with h5py.File(found_file,'r') as f:
    print(f'file:{found_file}: {f}')
    # print(f.visititems)
    _out = f.visititems(pho_visit_item_fn)
    # group.visit(find_foo)

    # f.attrs
    # print_value_overview_only(f)
    # # found_data_dict = {a_key:meta_info_dict for a_key in list(f.keys()) if a_key not in ignored_keys} ## filter the ignored keys
    # # found_keys = [a_key for a_key in list(f.keys()) if a_key not in ignored_keys] ## filter the ignored keys
    # f.name

## QFileSystemModel, QTreeView: Fresh Implementation (NOT USED)

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QFileSystemModel, QTreeView, QWidget, QVBoxLayout
from PyQt5.QtGui import QIcon

class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'PyQt5 file system view - pythonspot.com'
        self.left = 10
        self.top = 10
        self.width = 640
        self.height = 480
        self.initUI()
    
    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)
        
        self.model = QFileSystemModel()
        self.model.setRootPath(r'E:\data')
        self.tree = QTreeView()
        self.tree.setModel(self.model)
        
        self.tree.setAnimated(False)
        self.tree.setIndentation(20)
        self.tree.setSortingEnabled(True)
        
        self.tree.setWindowTitle("Dir View")
        self.tree.resize(640, 480)
        
        windowLayout = QVBoxLayout()
        windowLayout.addWidget(self.tree)
        self.setLayout(windowLayout)
        
        self.show()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())